# Magic Formula

In [1]:
import numpy as np
import pandas as pd
import string
import warnings

warnings.filterwarnings('ignore')

import requests

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [4]:
#na tabela do site fundamentus o decimal é colocado como virgula
df = pd.read_html(r.text,  decimal=',', thousands='.')[0] 

In [5]:
df[:5]

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MNSA3,0.42,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,...,0.0,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
1,CSTB4,147.69,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,...,0.0,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
2,POPR4,10.17,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,...,0.0,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
3,CFLU4,1000.00,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,...,0.0,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
4,PMET3,0.00,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,...,0.0,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"


In [6]:
#Os indicadores com % são raspados como strings
#Este for transforma eles em float
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
    df[coluna] = df[coluna].str.replace('.', '')
    df[coluna] = df[coluna].str.replace(',', '.')
    df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

## Analisando os dados 

In [7]:
#liquidez diária 
df = df[df['Liq.2meses'] > 1000000]
df.shape

(240, 21)

In [8]:
df.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [9]:
df[:5]

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
16,HBSA3,2.68,-249.64,1.53,1.153,0.0,0.320,3.41,6.99,-0.54,...,9.03,0.1649,-0.0046,1.86,0.0536,-0.0061,10936400.0,1.330800e+09,3.42,0.1952
24,SMFT3,16.20,-106.20,2.27,8.786,0.0,0.788,5.37,-87.69,-2.20,...,12.59,-0.1002,-0.0827,2.01,-0.0122,-0.0214,33561400.0,4.186390e+09,0.82,-0.0287
32,BRKM5,23.76,-56.43,2.59,0.196,0.0,0.215,1.84,3.10,-0.40,...,3.67,0.0633,-0.0085,1.42,0.1001,-0.0458,70363500.0,7.321800e+09,4.86,0.1878
35,ALPA4,8.60,-52.91,1.07,1.413,0.0,0.723,2.99,13.71,8.82,...,11.30,0.1031,-0.0269,2.50,0.0595,-0.0201,67925100.0,5.510850e+09,0.25,0.0230
38,MGLU3,3.81,-51.53,2.41,0.689,0.0,0.681,3.62,22.09,-4.96,...,12.93,0.0312,-0.0134,1.48,0.0456,-0.0469,396326000.0,1.064870e+10,0.67,0.2608


In [10]:
filtro = df['ROIC'] > 0.0
df = df[filtro]
filtro = df['Mrg. Líq.'] > 0.0
df = df[filtro]
filtro = df['P/L'] > 3.0
df = df[filtro]
filtro = df['ROE'] > 0.0
df = df[filtro]
filtro = df['EV/EBIT'] > 0.0
df = df[filtro]

In [11]:
df.shape

(164, 21)

In [12]:
num_acoes = df.shape[0]

## Rankings

In [13]:
rankingEBIT = pd.DataFrame()
rankingEBIT['pos_EBIT'] = range(1,num_acoes)
rankingROIC = pd.DataFrame()
rankingROIC['pos_ROIC'] = range(1,num_acoes)

In [14]:
#Este múltiplo compara o valor da empresa com o lucro antes de impostos e taxas (Ebit).
#EV/EBIT maior que zero - > Só lucro operacional positivo

rankingEBIT['Papel'] = df.sort_values(by=['EV/EBIT'])['Papel'][:(num_acoes-1)].values
rankingEBIT['EV/EBIT %'] = df.sort_values(by=['EV/EBIT'], ascending=True)['EV/EBIT'][:(num_acoes-1)].values

In [15]:
'''ROIC - Return on Invested Capital
significa o RETORNO SOBRE CAPITAL INVESTIDO. Na prática, ele informa quanto de dinheiro uma empresa consegue 
gerar em razão de todo o capital investido, incluindo os aportes por meio de dívidas.
'''

rankingROIC['Papel'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:(num_acoes-1)].values
rankingROIC['ROIC Value'] = df.sort_values(by=['ROIC'], ascending=False)['ROIC'][:(num_acoes-1)].values

In [16]:
rankingROIC[:5]

,pos_ROIC,Papel,ROIC Value
0,1,UNIP6,0.5872
1,2,UNIP3,0.5872
2,3,PSSA3,0.5742
3,4,KEPL3,0.4637
4,5,FESA4,0.3969


In [17]:
rankingEBIT

,pos_EBIT,Papel,EV/EBIT %
0,1,PSSA3,0.23
1,2,GOAU4,1.16
2,3,WIZC3,2.34
3,4,USIM3,2.52
4,5,USIM5,2.62
...,...,...,...
158,159,WEGE3,26.30
159,160,HBOR3,28.06
160,161,ENEV3,34.56
161,162,LWSA3,39.66


## SOMAR POSIÇÕES

In [18]:
ranking = pd.merge(rankingEBIT,rankingROIC)
ranking[:3]

,pos_EBIT,Papel,EV/EBIT %,pos_ROIC,ROIC Value
0,1,PSSA3,0.23,3,0.5742
1,2,GOAU4,1.16,18,0.2370
2,3,WIZC3,2.34,22,0.2203


In [19]:
ranking['pts'] = ranking["pos_EBIT"] + ranking["pos_ROIC"]
ranking[:3]

,pos_EBIT,Papel,EV/EBIT %,pos_ROIC,ROIC Value,pts
0,1,PSSA3,0.23,3,0.5742,4
1,2,GOAU4,1.16,18,0.2370,20
2,3,WIZC3,2.34,22,0.2203,25


In [20]:
def retirar_duplicados(rank):
    Ticker = rank['Papel'].str.extract(r'([A-Z]{4})')
    rank["Ticker"] = Ticker
    rank.set_index("Ticker",inplace=True)
    rank = rank[~rank.index.duplicated(keep='first')]
    rank.reset_index(inplace=True)
    rank.index = rank.index + 1
    rank.drop('Ticker',axis=1,inplace=True)
    return rank

def retirar_financeiros(rank):
    financeiros = ['PSSA3', 'BBDC4','SANB11', 'BBAS4','BBSE4','ITSA4','ITUB3','ITUB3','WIZC3', 'ABCB3', 'ABCB4']
    for f in financeiros:
        rank = rank[rank.Papel != f]
    rank.reset_index(inplace=True)
    rank.index = rank.index + 1
    rank.drop('index',axis=1,inplace=True)
    return rank

rank = ranking.sort_values('pts')
rank = rank[["Papel","EV/EBIT %","ROIC Value","pts"]]
rank = retirar_duplicados(rank)
rank = retirar_financeiros(rank)

rank.head(10)

,Papel,EV/EBIT %,ROIC Value,pts
1,UNIP3,2.63,0.5872,8
2,KEPL3,2.73,0.4637,11
3,TASA4,3.03,0.3800,17
4,FESA4,3.25,0.3969,17
5,GOAU4,1.16,0.2370,20
6,GGBR3,2.99,0.2373,27
7,RECV3,3.56,0.2880,28
8,PLPL3,4.30,0.3123,32
9,PRNR3,3.61,0.2219,39
10,VALE3,4.17,0.2142,48


### Salvar 

In [85]:
import csv
from datetime import datetime

data = datetime.today().strftime('%d-%m-%Y')
# abrindo o arquivo para escrita
rank.head(10).to_csv('magic_formula_{}.csv'.format(data))


## CALCULAR EARNING YIELD ALTERADO